In [49]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/zstikg/NewsConcept Data-set.json
/kaggle/input/zstikg/DVD playerData-set.json
/kaggle/input/zstikg/MedicalConcept Data-set.json
/kaggle/input/zstikg/Cellular phone Data-set.json
/kaggle/input/zstikg/Digital camera2 Data-set.json
/kaggle/input/zstikg/Mp3 playerData-set.json
/kaggle/input/zstikg/Digital camera1 Data-set.json


In [50]:
# ============================================================================
# PART 1: INSTALLATION AND SETUP
# ============================================================================

# Install required packages
!pip install dspy-ai huggingface_hub networkx sentence-transformers pandas --quiet

print("✓ Packages installed successfully!")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


✓ Packages installed successfully!


In [51]:
# ============================================================================
# PART 1: IMPORTS
# ============================================================================

import json
import networkx as nx
from typing import List, Dict, Set, Tuple
import dspy
from sentence_transformers import SentenceTransformer
import numpy as np
from collections import defaultdict
import pandas as pd
import os

print("✓ All imports successful!")

✓ All imports successful!


In [52]:
# ============================================================================
# STEP 1: INSTALL AND IMPORT (if not done already)
# ============================================================================

import os
from huggingface_hub import InferenceClient
import dspy
import json
from typing import List, Tuple, Dict

print("✓ Imports complete")

✓ Imports complete


In [53]:
# ============================================================================
# STEP 2: CONFIGURE HUGGING FACE WITH DSPY.LM (PROPER WAY)
# ============================================================================
import os
import dspy

os.environ['HUGGINGFACE_API_KEY'] = 'KEY'

# Use DSPy's built-in LM class with the correct prefix
lm = dspy.LM(
    model='huggingface/meta-llama/Llama-3.1-8B-Instruct',  # Add 'huggingface/' prefix!
    api_key=os.environ['HUGGINGFACE_API_KEY'],
    max_tokens=2000,
    temperature=0.3
)

dspy.settings.configure(lm=lm)
print("✓ Llama-3.1-8B-Instruct configured successfully!")

✓ Llama-3.1-8B-Instruct configured successfully!


In [54]:
# ============================================================================
# PART 1: DATA LOADING FUNCTION
# ============================================================================

def load_json_data(filepath: str) -> List[Dict]:
    """
    Load JSON data from file
    
    Expected format:
    [
        {
            "Article Title": [],
            "Article Text": "text here...",
            "Concept": ["concept1", "concept2"]
        },
        ...
    ]
    """
    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    print(f"✓ Loaded {len(data)} documents from {filepath}")
    return data

# Test with sample data
sample_data = [
    {
        "Article Title": [],
        "Article Text": "excellent phone, excellent service.",
        "Concept": []
    },
    {
        "Article Title": [],
        "Article Text": "i am a business user who heavily depend on mobile service.",
        "Concept": ["service"]
    }
]

print("✓ Sample data ready for testing")
print(f"  Sample has {len(sample_data)} documents")

✓ Sample data ready for testing
  Sample has 2 documents


In [55]:
# ============================================================================
# FIND THE DATASET FILES
# ============================================================================

import os

# List files in your dataset directory
dataset_path = '/kaggle/input/zstikg'

print("Files in the dataset:")
print("=" * 60)

try:
    files = os.listdir(dataset_path)
    for i, file in enumerate(files, 1):
        print(f"{i}. {file}")
        full_path = os.path.join(dataset_path, file)
        print(f"   Full path: {full_path}")
        print()
except Exception as e:
    print(f"Error: {e}")
    print("\nTrying to list all datasets...")
    for dataset in os.listdir('/kaggle/input/'):
        print(f"- {dataset}")

Files in the dataset:
1. NewsConcept Data-set.json
   Full path: /kaggle/input/zstikg/NewsConcept Data-set.json

2. DVD playerData-set.json
   Full path: /kaggle/input/zstikg/DVD playerData-set.json

3. MedicalConcept Data-set.json
   Full path: /kaggle/input/zstikg/MedicalConcept Data-set.json

4. Cellular phone Data-set.json
   Full path: /kaggle/input/zstikg/Cellular phone Data-set.json

5. Digital camera2 Data-set.json
   Full path: /kaggle/input/zstikg/Digital camera2 Data-set.json

6. Mp3 playerData-set.json
   Full path: /kaggle/input/zstikg/Mp3 playerData-set.json

7. Digital camera1 Data-set.json
   Full path: /kaggle/input/zstikg/Digital camera1 Data-set.json



In [56]:
# ============================================================================
# LOAD ONE SPECIFIC DATASET
# ============================================================================

# Choose which dataset you want to work with:
# Option 1: Cellular phone
filepath = '/kaggle/input/zstikg/MedicalConcept Data-set.json'

# Option 2: News
# filepath = '/kaggle/input/zsltikg/NewsConcept Data-set.json'

# Option 3: Medical
# filepath = '/kaggle/input/zsltikg/MedicalConcept Data-set.json'

# Load the data
data = load_json_data(filepath)

print(f"✓ Loaded {len(data)} documents")
print(f"\nFirst document preview:")
print(f"  Keys: {list(data[0].keys())}")
print(f"  Text: {data[0].get('Article Text', '')[:100]}...")
print(f"  Concepts: {data[0].get('Concept', [])}")

✓ Loaded 2066 documents from /kaggle/input/zstikg/MedicalConcept Data-set.json
✓ Loaded 2066 documents

First document preview:
  Keys: ['Article Title', 'Article Text', 'Concept']
  Text: 1. Health care reform

How could the health care reform legislation that President Barack Obama sign...
  Concepts: []


In [57]:
# ============================================================================
# RELOAD DATA PROPERLY
# ============================================================================

import json

print("=" * 80)
print("RELOADING DATA PROPERLY")
print("=" * 80)

# Load the JSON file correctly
with open('/kaggle/input/zstikg/MedicalConcept Data-set.json', 'r') as f:
    raw_data = json.load(f)

print(f"Type after loading: {type(raw_data)}")

# Check if it's a dict with a key
if isinstance(raw_data, dict):
    print(f"Keys: {raw_data.keys()}")
    # Get the actual data
    for key in raw_data.keys():
        if isinstance(raw_data[key], list):
            data = raw_data[key]
            print(f"Found list under key '{key}' with {len(data)} items")
            break
elif isinstance(raw_data, list):
    data = raw_data
    print(f"Data is a list with {len(data)} items")

# Verify first item
print(f"\nFirst item type: {type(data[0])}")
#print(f"First item: {data[0]}")

RELOADING DATA PROPERLY
Type after loading: <class 'list'>
Data is a list with 2066 items

First item type: <class 'dict'>


In [58]:
# ============================================================================
# PROCESS YOUR ALL THE ARTICLES
# ============================================================================

print("=" * 80)
print("PROCESSING ALL THE ARTICLES")
print("=" * 80)

# data is now correctly loaded as a list of dicts
individual_articles = []

for idx, item in enumerate(data):
    article = {
        'id': idx,
        'Article Text': item['Article Text'],
        'Concept': item['Concept'] if item['Concept'] else []
    }
    individual_articles.append(article)

print(f"\n✓ Created list of {len(individual_articles)} individual articles")

# Show first 3
print(f"\nFirst 3 articles:")
print("-" * 80)
for i in range(min(3, len(individual_articles))):
    article = individual_articles[i]
    print(f"\nArticle {article['id']}:")
    print(f"  Text: {article['Article Text'][:80]}...")
    print(f"  Concepts: {article['Concept']}")

PROCESSING ALL THE ARTICLES

✓ Created list of 2066 individual articles

First 3 articles:
--------------------------------------------------------------------------------

Article 0:
  Text: 1. Health care reform

How could the health care reform legislation that Preside...
  Concepts: []

Article 1:
  Text: My colleagues at Harvard Health Publishing and I have a mission: to provide accu...
  Concepts: ['Heart Health', "Women's Health"]

Article 2:
  Text: Does echinacea, the popular natural cold remedy, really work?

It depends on wha...
  Concepts: []


In [59]:
# ============================================================================
# STEP 3: DEFINE SIGNATURES
# ============================================================================

class EntityExtractor(dspy.Signature):
    """Extract key entities from the given text. Extracted entities are nouns, 
    verbs, or adjectives, particularly regarding sentiment. This is for an 
    extraction task, please be thorough and accurate to the reference text.
    
    Return ONLY a valid JSON list format: ["entity1", "entity2", "entity3"]
    """
    
    text = dspy.InputField(desc="The text to extract entities from")
    entities = dspy.OutputField(desc="List of extracted entities in JSON format")

class RelationExtractor(dspy.Signature):
    """Extract subject-predicate-object triples from the assistant message. 
    A predicate (1-3 words) defines the relationship between the subject and 
    object. Relationship may be fact or sentiment based on assistant's message. 
    Subject and object are entities. Entities provided are from the assistant 
    message and prior conversation history, though you may not need all of them. 
    This is for an extraction task, please be thorough, accurate, and faithful 
    to the reference text.
    
    Return ONLY valid JSON format: [["subject1", "predicate1", "object1"], ["subject2", "predicate2", "object2"]]
    """
    
    text = dspy.InputField(desc="The text to extract relations from")
    entities = dspy.InputField(desc="List of available entities")
    triples = dspy.OutputField(desc="List of [subject, predicate, object] triples in JSON format")

print("✓ Signatures defined")

✓ Signatures defined


In [60]:
# ============================================================================
# STEP 4: CREATE ENTITY EXTRACTOR
# ============================================================================

class ExtractEntities(dspy.Module):
    def __init__(self):
        super().__init__()
        self.extract = dspy.ChainOfThought(EntityExtractor)
    
    def forward(self, text: str) -> List[str]:
        if not text or len(text.strip()) < 3:
            return []
            
        result = self.extract(text=text)
        
        try:
            entities_text = result.entities.strip()
            
            if '[' in entities_text and ']' in entities_text:
                start = entities_text.find('[')
                end = entities_text.rfind(']') + 1
                entities_text = entities_text[start:end]
            
            entities = json.loads(entities_text)
            
            if isinstance(entities, list):
                return [str(e).lower().strip() for e in entities if e and len(str(e).strip()) > 1]
            return []
            
        except:
            try:
                entities_text = result.entities.strip()
                if entities_text.startswith('['):
                    entities_text = entities_text[1:]
                if entities_text.endswith(']'):
                    entities_text = entities_text[:-1]
                
                entities = []
                for item in entities_text.split(','):
                    item = item.strip(' "\'\n\t')
                    if item and len(item) > 1:
                        entities.append(item.lower())
                
                return entities[:50]
            except:
                return []

entity_extractor = ExtractEntities()
print("✓ Entity Extractor created")

✓ Entity Extractor created


In [61]:
# ============================================================================
# STEP 5: CREATE RELATION EXTRACTOR
# ============================================================================

class ExtractRelations(dspy.Module):
    def __init__(self):
        super().__init__()
        self.extract = dspy.ChainOfThought(RelationExtractor)
    
    def forward(self, text: str, entities: List[str]) -> List[Tuple[str, str, str]]:
        if not entities or not text:
            return []
        
        entities_subset = entities[:30]
        entities_str = json.dumps(entities_subset)
        
        result = self.extract(text=text, entities=entities_str)
        
        try:
            triples_text = result.triples.strip()
            
            if '[' in triples_text and ']' in triples_text:
                start = triples_text.find('[')
                end = triples_text.rfind(']') + 1
                triples_text = triples_text[start:end]
            
            triples = json.loads(triples_text)
            
            normalized_triples = []
            for triple in triples:
                if isinstance(triple, (list, tuple)) and len(triple) == 3:
                    s, p, o = triple
                    s = str(s).lower().strip()
                    p = str(p).lower().strip()
                    o = str(o).lower().strip()
                    
                    if s and p and o and s != o:
                        normalized_triples.append((s, p, o))
            
            return normalized_triples[:100]
            
        except Exception as e:
            return []

relation_extractor = ExtractRelations()
print("✓ Relation Extractor created")

✓ Relation Extractor created


In [62]:
# ============================================================================
# STEP 6: TEST EXTRACTION
# ============================================================================

print("\nTesting Entity & Relation Extraction...")
print("=" * 80)

# Test on first 2 documents
for i in range(min(2, len(data))):
    text = data[i].get('Article Text', '')
    
    print(f"\nDocument {i+1}:")
    print(f"  Text: {text[:80]}...")
    
    # Extract entities
    print(f"  Extracting entities...")
    entities = entity_extractor(text)
    print(f"  ✓ Found {len(entities)} entities: {entities[:5]}...")
    
    # Extract relations
    if entities:
        print(f"  Extracting relations...")
        relations = relation_extractor(text, entities)
        print(f"  ✓ Found {len(relations)} relations")
        
        if relations:
            for j, (s, p, o) in enumerate(relations[:3], 1):
                print(f"    {j}. ({s}) --[{p}]--> ({o})")

print("\n" + "=" * 80)
print("✓ Test complete!")


Testing Entity & Relation Extraction...

Document 1:
  Text: 1. Health care reform

How could the health care reform legislation that Preside...
  Extracting entities...
  ✓ Found 36 entities: ['health care reform', 'president barack obama', 'patient protection and affordable care act', 'medicaid', 'health insurance']...
  Extracting relations...
  ✓ Found 13 relations
    1. (president barack obama) --[signed into law]--> (patient protection and affordable care act)
    2. (patient protection and affordable care act) --[designed to]--> (patch holes in the health insurance system)
    3. (patient protection and affordable care act) --[extend coverage to]--> (32 million americans)

Document 2:
  Text: My colleagues at Harvard Health Publishing and I have a mission: to provide accu...
  Extracting entities...
  ✓ Found 7 entities: ['harvard health publishing', "harvard women's health watch", 'harvard heart letter', "st. joseph's hospital", 'tim russert']...
  Extracting relations...
  ✓

In [63]:
# ============================================================================
# CLUSTERING SIGNATURES (DEFINE FIRST!)
# ============================================================================

class ClusterValidator(dspy.Signature):
    """Verify if these entities belong in the same cluster.
    A cluster should contain entities that are the same in meaning, with different:
    - tenses, plural forms, stem forms, upper/lower cases
    Or entities with close semantic meanings.
    
    Return ONLY valid JSON format: ["entity1", "entity2", "entity3"]
    Return only entities you are confident belong together.
    If not confident, return empty list [].
    """
    
    entities = dspy.InputField(desc="Entities to validate")
    valid_cluster = dspy.OutputField(desc="Validated cluster in JSON format")

print("✓ ClusterValidator Signature defined")

✓ ClusterValidator Signature defined


In [64]:
# ============================================================================
# SEMANTIC SIMILARITY CLUSTERING (FROM PAPER)
# ============================================================================

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

class SemanticEntityClustering(dspy.Module):
    def __init__(self, similarity_threshold=0.75):
        super().__init__()
        self.validator = dspy.ChainOfThought(ClusterValidator)
        
        # Load embedding model (same as paper)
        print("Loading sentence transformer model...")
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        print("✓ Model loaded")
        
        self.similarity_threshold = similarity_threshold
    
    def _parse_cluster(self, text: str) -> List[str]:
        """Parse cluster from LLM response"""
        try:
            text = text.strip()
            if '[' in text and ']' in text:
                start = text.find('[')
                end = text.rfind(']') + 1
                text = text[start:end]
            
            cluster = json.loads(text)
            if isinstance(cluster, list):
                return [str(e).lower().strip() for e in cluster if e]
            return []
        except:
            return []
    
    def _get_semantic_clusters(self, entities: List[str]) -> List[List[str]]:
        """Group entities by semantic similarity using embeddings"""
        
        if len(entities) == 0:
            return []
        
        # Get embeddings for all entities
        embeddings = self.model.encode(entities)
        
        # Compute pairwise cosine similarity
        similarity_matrix = cosine_similarity(embeddings)
        
        # Find clusters using similarity threshold
        clusters = []
        remaining = set(range(len(entities)))
        
        for i in range(len(entities)):
            if i not in remaining:
                continue
            
            # Find all entities similar to this one
            cluster_indices = [i]
            remaining.discard(i)
            
            for j in range(i + 1, len(entities)):
                if j not in remaining:
                    continue
                
                # Check if similar enough
                if similarity_matrix[i][j] >= self.similarity_threshold:
                    cluster_indices.append(j)
                    remaining.discard(j)
            
            # Convert indices to entity names
            cluster = [entities[idx] for idx in cluster_indices]
            
            # Only keep clusters with 2-4 entities
            if 2 <= len(cluster) <= 4:
                clusters.append(cluster)
            elif len(cluster) == 1:
                # Keep singletons for later
                pass
        
        return clusters
    
    def forward(self, entities: List[str]) -> Dict[str, List[str]]:
        """Semantic clustering with LLM validation"""
        
        print(f"Starting semantic clustering with {len(entities)} entities...")
        print(f"  Similarity threshold: {self.similarity_threshold}")
        
        # Remove duplicates
        unique_entities = list(set(entities))
        
        # Step 1: Find semantic clusters using embeddings
        print("  Computing semantic similarities...")
        potential_clusters = self._get_semantic_clusters(unique_entities)
        
        print(f"  Found {len(potential_clusters)} potential clusters")
        
        # Step 2: Validate with LLM
        validated_clusters = {}
        cluster_id = 0
        clustered_entities = set()
        
        for cluster in potential_clusters:
            try:
                # Ask LLM to validate
                validation = self.validator(entities=json.dumps(cluster))
                validated = self._parse_cluster(validation.valid_cluster)
                
                if validated and len(validated) >= 2:
                    cluster_label = validated[0]
                    validated_clusters[cluster_label] = validated
                    
                    for entity in validated:
                        clustered_entities.add(entity)
                    
                    print(f"  ✓ Cluster {cluster_id}: {validated}")
                    cluster_id += 1
                else:
                    # LLM rejected - add as singletons
                    for entity in cluster:
                        if entity not in clustered_entities:
                            validated_clusters[entity] = [entity]
                            clustered_entities.add(entity)
            except:
                # Error - add as singletons
                for entity in cluster:
                    if entity not in clustered_entities:
                        validated_clusters[entity] = [entity]
                        clustered_entities.add(entity)
        
        # Step 3: Add all remaining entities as singletons
        for entity in unique_entities:
            if entity not in clustered_entities:
                validated_clusters[entity] = [entity]
        
        multi = sum(1 for v in validated_clusters.values() if len(v) > 1)
        print(f"✓ Semantic clustering complete: {len(validated_clusters)} total clusters")
        print(f"  Multi-entity clusters: {multi}")
        print(f"  Singleton entities: {len(validated_clusters) - multi}")
        
        return validated_clusters

# Create semantic clusterer with different thresholds
entity_clusterer_semantic = SemanticEntityClustering(similarity_threshold=0.75)
print("\n✓ Semantic Entity Clustering Module created")

Loading sentence transformer model...
✓ Model loaded

✓ Semantic Entity Clustering Module created


In [65]:
# ============================================================================
# TEST SEMANTIC CLUSTERING
# ============================================================================

print("\nTesting SEMANTIC Clustering (Embeddings + LLM)...")
print("=" * 80)

test_entities = [
    # Should cluster (same concept)
    'phone', 'mobile phone', 'cellphone', 'cell phone',
    
    # Should cluster (same concept)
    'usa', 'united states', 'america',
    
    # Should cluster (similar meaning)
    'helping hand', 'assistant', 'helper', 'support',
    
    # Should cluster (same concept)
    'service', 'services',
    
    # Should cluster (same concept)
    'call', 'calling', 'calls',
    
    # Should NOT cluster (different brands)
    'nokia', 'motorola', 'samsung',
    
    # Should NOT cluster (different concepts)
    'signal', 'battery', 'screen',
]

print(f"Testing with {len(test_entities)} entities:")
for e in test_entities:
    print(f"  - {e}")

print("\n" + "=" * 80)

# Cluster with semantic method
semantic_clusters = entity_clusterer_semantic(test_entities)

print("\n" + "=" * 80)
print("SEMANTIC CLUSTERING RESULTS:")
print("=" * 80)

# Show multi-entity clusters
multi_clusters = {k: v for k, v in semantic_clusters.items() if len(v) > 1}

if multi_clusters:
    print(f"\n✓ Found {len(multi_clusters)} semantic clusters:")
    for i, (label, members) in enumerate(multi_clusters.items(), 1):
        print(f"  {i}. {members}")
        
        # Show similarity scores
        if len(members) > 1:
            embeddings = entity_clusterer_semantic.model.encode(members)
            similarities = cosine_similarity(embeddings)
            avg_sim = (similarities.sum() - len(members)) / (len(members) * (len(members) - 1))
            print(f"     Average similarity: {avg_sim:.3f}")
else:
    print("\n  No clusters found")

print(f"\nSingleton entities: {len(semantic_clusters) - len(multi_clusters)}")

print("\n✓ Test complete!")


Testing SEMANTIC Clustering (Embeddings + LLM)...
Testing with 22 entities:
  - phone
  - mobile phone
  - cellphone
  - cell phone
  - usa
  - united states
  - america
  - helping hand
  - assistant
  - helper
  - support
  - service
  - services
  - call
  - calling
  - calls
  - nokia
  - motorola
  - samsung
  - signal
  - battery
  - screen

Starting semantic clustering with 22 entities...
  Similarity threshold: 0.75
  Computing semantic similarities...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

  Found 4 potential clusters
  ✓ Cluster 0: ['phone', 'cell phone', 'mobile phone', 'cellphone']
  ✓ Cluster 1: ['call', 'call', 'calls']
  ✓ Cluster 2: ['america', 'united states', 'usa']
  ✓ Cluster 3: ['service', 'services']
✓ Semantic clustering complete: 15 total clusters
  Multi-entity clusters: 4
  Singleton entities: 11

SEMANTIC CLUSTERING RESULTS:

✓ Found 4 semantic clusters:
  1. ['phone', 'cell phone', 'mobile phone', 'cellphone']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

     Average similarity: 0.860
  2. ['call', 'call', 'calls']


...

     Average similarity: 0.894
  3. ['america', 'united states', 'usa']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

     Average similarity: 0.844
  4. ['service', 'services']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

     Average similarity: 0.921

Singleton entities: 11

✓ Test complete!


In [67]:
# ============================================================================
# VISUALIZE HOW CLUSTERING WORKS
# ============================================================================

from sklearn.metrics.pairwise import cosine_similarity

# Test entities
test_entities = ['phone', 'mobile phone', 'cellphone', 'nokia', 'motorola']

# Get embeddings
embeddings = entity_clusterer_semantic.model.encode(test_entities)

# Calculate similarities
print("Similarity Matrix:")
print("=" * 80)
print(f"{'':15s}", end='')
for e in test_entities:
    print(f"{e:15s}", end='')
print()

for i, entity1 in enumerate(test_entities):
    print(f"{entity1:15s}", end='')
    for j, entity2 in enumerate(test_entities):
        similarity = cosine_similarity([embeddings[i]], [embeddings[j]])[0][0]
        print(f"{similarity:15.3f}", end='')
    print()

print("\n" + "=" * 80)
print(f"Threshold: 0.75")
print("=" * 80)

# Show which pairs would cluster
print("\nPairs above threshold (would cluster together):")
for i, entity1 in enumerate(test_entities):
    for j, entity2 in enumerate(test_entities):
        if i < j:  # Avoid duplicates
            similarity = cosine_similarity([embeddings[i]], [embeddings[j]])[0][0]
            if similarity >= 0.75:
                print(f"  ✓ {entity1:15s} ↔ {entity2:15s} = {similarity:.3f}")

print("\nPairs below threshold (stay separate):")
for i, entity1 in enumerate(test_entities):
    for j, entity2 in enumerate(test_entities):
        if i < j:
            similarity = cosine_similarity([embeddings[i]], [embeddings[j]])[0][0]
            if similarity < 0.75:
                print(f"  ✗ {entity1:15s} ↔ {entity2:15s} = {similarity:.3f}")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Similarity Matrix:
               phone          mobile phone   cellphone      nokia          motorola       
phone                    1.000          0.814          0.845          0.681          0.644
mobile phone             0.814          1.000          0.843          0.751          0.688
cellphone                0.845          0.843          1.000          0.674          0.617
nokia                    0.681          0.751          0.674          1.000          0.640
motorola                 0.644          0.688          0.617          0.640          1.000

Threshold: 0.75

Pairs above threshold (would cluster together):
  ✓ phone           ↔ mobile phone    = 0.814
  ✓ phone           ↔ cellphone       = 0.845
  ✓ mobile phone    ↔ cellphone       = 0.843
  ✓ mobile phone    ↔ nokia           = 0.751

Pairs below threshold (stay separate):
  ✗ phone           ↔ nokia           = 0.681
  ✗ phone           ↔ motorola        = 0.644
  ✗ mobile phone    ↔ motorola        = 0.688
  ✗ cel

In [68]:
# ============================================================================
# FINAL KGGEN WITH SEMANTIC CLUSTERING
# ============================================================================

class KGGenSemantic:
    def __init__(self, similarity_threshold=0.75):
        self.entity_extractor = entity_extractor
        self.relation_extractor = relation_extractor
        self.entity_clusterer = SemanticEntityClustering(similarity_threshold=similarity_threshold)
        self.graph = nx.DiGraph()
        self.entity_clusters = {}
        
    def generate_from_json(self, json_data: List[Dict], max_docs: int = None) -> nx.DiGraph:
        """Generate KG from JSON dataset"""
        all_entities = set()
        all_relations = []
        
        if max_docs:
            json_data = json_data[:max_docs]
        
        print(f"Processing {len(json_data)} documents...")
        print("=" * 80)
        
        for idx, item in enumerate(json_data):
            text = item.get('Article Text', '')
            concepts = item.get('Concept', [])
            
            if not text or len(text.strip()) < 5:
                continue
            
            try:
                # Extract entities
                entities = self.entity_extractor(text)
                all_entities.update(entities)
                
                # Add concepts
                for concept in concepts:
                    if concept and isinstance(concept, str):
                        all_entities.add(concept.lower().strip())
                
                # Extract relations
                relations = self.relation_extractor(text, list(all_entities))
                all_relations.extend(relations)
                
                if (idx + 1) % 20 == 0:
                    print(f"  {idx + 1}/{len(json_data)} docs | {len(all_entities)} entities | {len(all_relations)} relations")
                    
            except Exception as e:
                continue
        
        print(f"\n✓ Extraction complete!")
        print(f"  Total entities: {len(all_entities)}")
        print(f"  Total relations: {len(all_relations)}")
        
        # Build graph
        for subj, pred, obj in all_relations:
            self.graph.add_edge(subj, obj, relation=pred)
        
        print(f"  Graph nodes: {len(self.graph.nodes())}")
        print(f"  Graph edges: {len(self.graph.edges())}")
        
        return self.graph
    
    def cluster_entities(self):
        """Semantic clustering with embeddings"""
        nodes = list(self.graph.nodes())
        
        if len(nodes) == 0:
            print("No nodes to cluster!")
            return self.graph
        
        print(f"\n{'='*80}")
        print(f"SEMANTIC CLUSTERING: {len(nodes)} ENTITIES")
        print(f"{'='*80}")
        
        self.entity_clusters = self.entity_clusterer(nodes)
        
        # Map entities
        entity_mapping = {}
        for cluster_label, cluster_entities in self.entity_clusters.items():
            for entity in cluster_entities:
                entity_mapping[entity] = cluster_label
        
        # Rebuild graph
        new_graph = nx.DiGraph()
        for u, v, data in self.graph.edges(data=True):
            new_u = entity_mapping.get(u, u)
            new_v = entity_mapping.get(v, v)
            relation = data.get('relation', 'related_to')
            
            if new_u == new_v:
                continue
            
            if not new_graph.has_edge(new_u, new_v):
                new_graph.add_edge(new_u, new_v, relation=relation)
        
        self.graph = new_graph
        
        print(f"\n✓ Clustering complete!")
        print(f"  Final nodes: {len(self.graph.nodes())}")
        print(f"  Final edges: {len(self.graph.edges())}")
        
        return self.graph
    
    def save_graph(self, filepath: str):
        data = nx.node_link_data(self.graph)
        with open(filepath, 'w') as f:
            json.dump(data, f, indent=2)
        print(f"✓ Saved to {filepath}")
    
    def export_triples(self, filepath: str):
        triples = []
        for u, v, data in self.graph.edges(data=True):
            triples.append({
                'subject': u,
                'predicate': data.get('relation', 'related_to'),
                'object': v
            })
        import pandas as pd
        df = pd.DataFrame(triples)
        df.to_csv(filepath, index=False)
        print(f"✓ Exported to {filepath}")

# Initialize semantic KGGen (threshold 0.75 = balanced)
kg_gen_semantic = KGGenSemantic(similarity_threshold=0.75)
print("\n✓ KGGen with Semantic Clustering (Embeddings + LLM) created!")

Loading sentence transformer model...
✓ Model loaded

✓ KGGen with Semantic Clustering (Embeddings + LLM) created!


In [70]:
# ============================================================================
# GENERATE KG FOR ALL THE ARTICLES
# ============================================================================

print("\n" + "=" * 80)
print("GENERATING KNOWLEDGE GRAPHS FOR ALL THE ARTICLES")
print("=" * 80)

all_article_kgs = []

total = len(individual_articles)
print(f"\nProcessing {total} articles...\n")

for idx, article in enumerate(individual_articles):
    article_id = article['id']
    text = article['Article Text']
    concepts = article['Concept']
    
    # Skip empty articles
    if not text or len(text.strip()) < 5:
        all_article_kgs.append({
            'id': article_id,
            'text': text,
            'concepts': concepts,
            'graph': nx.DiGraph(),
            'entities': [],
            'relations': [],
            'num_nodes': 0,
            'num_edges': 0
        })
        continue
    
    try:
        # Extract entities for THIS article
        entities = entity_extractor(text)
        
        # Add original concepts as entities
        for concept in concepts:
            if concept and isinstance(concept, str):
                entities.append(concept.lower().strip())
        
        entities = list(set(entities))  # Remove duplicates
        
        # Extract relations for THIS article
        if entities:
            relations = relation_extractor(text, entities)
        else:
            relations = []
        
        # Build graph for THIS article
        graph = nx.DiGraph()
        for subj, pred, obj in relations:
            graph.add_edge(subj, obj, relation=pred)
        
        # Store everything for this article
        all_article_kgs.append({
            'id': article_id,
            'text': text,
            'concepts': concepts,
            'graph': graph,
            'entities': entities,
            'relations': relations,
            'num_nodes': len(graph.nodes()),
            'num_edges': len(graph.edges())
        })
        
        # Progress update
        if (idx + 1) % 50 == 0:
            print(f"✓ Processed {idx + 1}/{total} articles...")
        
    except Exception as e:
        print(f"✗ Article {article_id}: Error - {str(e)[:100]}")
        all_article_kgs.append({
            'id': article_id,
            'text': text,
            'concepts': concepts,
            'graph': nx.DiGraph(),
            'entities': [],
            'relations': [],
            'num_nodes': 0,
            'num_edges': 0
        })

print(f"\n" + "=" * 80)
print("✓ KNOWLEDGE GRAPH GENERATION COMPLETE!")
print("=" * 80)
print(f"Total articles processed: {len(all_article_kgs)}")
print(f"Articles with graphs: {sum(1 for kg in all_article_kgs if kg['num_edges'] > 0)}")
print(f"Articles without graphs: {sum(1 for kg in all_article_kgs if kg['num_edges'] == 0)}")

# Statistics
total_entities = sum(len(kg['entities']) for kg in all_article_kgs)
total_relations = sum(len(kg['relations']) for kg in all_article_kgs)

print(f"\nTotal entities extracted: {total_entities}")
print(f"Total relations extracted: {total_relations}")

with_graphs = [kg for kg in all_article_kgs if kg['num_edges'] > 0]
if with_graphs:
    avg_nodes = sum(kg['num_nodes'] for kg in with_graphs) / len(with_graphs)
    avg_edges = sum(kg['num_edges'] for kg in with_graphs) / len(with_graphs)
    print(f"\nAverage nodes per KG: {avg_nodes:.2f}")
    print(f"Average edges per KG: {avg_edges:.2f}")


GENERATING KNOWLEDGE GRAPHS FOR ALL THE ARTICLES

Processing 2066 articles...

✓ Processed 50/2066 articles...


2025/10/22 17:31:48 WARNING dspy.clients.lm: LM response was truncated due to exceeding max_tokens=2000. You can inspect the latest LM interactions with `dspy.inspect_history()`. To avoid truncation, consider passing a larger max_tokens when setting up dspy.LM. You may also consider increasing the temperature (currently 0.3)  if the reason for truncation is repetition.


✓ Processed 100/2066 articles...
✓ Processed 150/2066 articles...


KeyboardInterrupt: 

In [71]:
# ============================================================================
# CHECK WHAT'S BEEN SUCCESSFULLY PROCESSED
# ============================================================================

print("=" * 80)
print("CHECKING SUCCESSFUL PROCESSING")
print("=" * 80)

# Count successes
successful = [kg for kg in all_article_kgs if kg['num_edges'] > 0]
failed = [kg for kg in all_article_kgs if kg['num_edges'] == 0]

print(f"\nTotal articles: {len(all_article_kgs)}")
print(f"✓ Successfully processed: {len(successful)}")
print(f"✗ Failed/Empty: {len(failed)}")

if successful:
    print(f"\nSuccessful articles with KGs:")
    for kg in successful[:10]:
        print(f"  Article {kg['id']}: {kg['num_nodes']} nodes, {kg['num_edges']} edges")
    
    # Export the successful ones
    print("\n" + "=" * 80)
    print("EXPORTING SUCCESSFUL RESULTS")
    print("=" * 80)
    
    import json
    import os
    
    # Create output directory
    os.makedirs('/kaggle/working/individual_kgs', exist_ok=True)
    
    # Export successful KGs
    for kg in successful:
        article_id = kg['id']
        
        graph_data = {
            'article_id': article_id,
            'text': kg['text'],
            'concepts': kg['concepts'],
            'entities': kg['entities'],
            'relations': [
                {'subject': s, 'predicate': p, 'object': o}
                for s, p, o in kg['relations']
            ],
            'graph': nx.node_link_data(kg['graph'])
        }
        
        filepath = f'/kaggle/working/individual_kgs/article_{article_id}.json'
        with open(filepath, 'w') as f:
            json.dump(graph_data, f, indent=2)
    
    print(f"✓ Saved {len(successful)} JSON files")
    
    # Export triples
    all_triples = []
    for kg in successful:
        for subj, pred, obj in kg['relations']:
            all_triples.append({
                'article_id': kg['id'],
                'subject': subj,
                'predicate': pred,
                'object': obj
            })
    
    triples_df = pd.DataFrame(all_triples)
    triples_df.to_csv('/kaggle/working/successful_triples.csv', index=False)
    print(f"✓ Saved {len(all_triples)} triples to successful_triples.csv")
    
    # Summary
    summary = []
    for kg in all_article_kgs:
        summary.append({
            'article_id': kg['id'],
            'text_preview': kg['text'][:100],
            'num_entities': len(kg['entities']),
            'num_relations': len(kg['relations']),
            'num_nodes': kg['num_nodes'],
            'num_edges': kg['num_edges'],
            'status': 'success' if kg['num_edges'] > 0 else 'failed'
        })
    
    summary_df = pd.DataFrame(summary)
    summary_df.to_csv('/kaggle/working/processing_summary.csv', index=False)
    print(f"✓ Saved summary")
    
    print("\n✓ DONE! Downloaded what we could process.")
else:
    print("\n✗ No successful KGs to export")

CHECKING SUCCESSFUL PROCESSING

Total articles: 151
✓ Successfully processed: 149
✗ Failed/Empty: 2

Successful articles with KGs:
  Article 0: 35 nodes, 23 edges
  Article 1: 22 nodes, 15 edges
  Article 2: 11 nodes, 9 edges
  Article 3: 31 nodes, 18 edges
  Article 4: 26 nodes, 21 edges
  Article 5: 23 nodes, 16 edges
  Article 6: 23 nodes, 18 edges
  Article 7: 20 nodes, 14 edges
  Article 8: 16 nodes, 11 edges
  Article 9: 21 nodes, 18 edges

EXPORTING SUCCESSFUL RESULTS
✓ Saved 149 JSON files
✓ Saved 1955 triples to successful_triples.csv
✓ Saved summary

✓ DONE! Downloaded what we could process.


/usr/local/lib/python3.11/dist-packages/networkx/readwrite/json_graph/node_link.py:145: FutureWarning: 
The default value will be `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_data(G, edges="links") to preserve current behavior, or
  nx.node_link_data(G, edges="edges") for forward compatibility.
  warnings.warn(


In [73]:
# ============================================================================
# VIEW SUCCESSFUL RESULTS
# ============================================================================

import pandas as pd

print("=" * 80)
print("YOUR 46 SUCCESSFUL KNOWLEDGE GRAPHS")
print("=" * 80)

# Load summary
summary_df = pd.read_csv('/kaggle/working/processing_summary.csv')
successful_df = summary_df[summary_df['status'] == 'success'].copy()

print(f"\nTop 10 Largest KGs:")
print("-" * 80)
top_kgs = successful_df.nlargest(10, 'num_edges')
print(top_kgs[['article_id', 'num_nodes', 'num_edges', 'text_preview']].to_string(index=False))

# Load triples
triples_df = pd.read_csv('/kaggle/working/successful_triples.csv')

print(f"\n\nSample Triples:")
print("-" * 80)
print(triples_df.head(20).to_string(index=False))

print(f"\n\nStatistics:")
print("-" * 80)
print(f"Total successful articles: {len(successful_df)}")
print(f"Total triples extracted: {len(triples_df)}")
print(f"Avg edges per KG: {successful_df['num_edges'].mean():.2f}")
print(f"Max edges in a KG: {successful_df['num_edges'].max()}")

print("\n" + "=" * 80)
print("FILES READY TO DOWNLOAD:")
print("=" * 80)
print("  📁 /kaggle/working/individual_kgs/ (46 JSON files)")
print("  📄 /kaggle/working/successful_triples.csv")
print("  📄 /kaggle/working/processing_summary.csv")

YOUR 46 SUCCESSFUL KNOWLEDGE GRAPHS

Top 10 Largest KGs:
--------------------------------------------------------------------------------
 article_id  num_nodes  num_edges                                                                                           text_preview
        147         45         31   Heat waves are unpleasant for healthy folks. For people with cardiovascular trouble, hazy, hot, humi
         67         37         28   A dear friend got caught up in a debate about these terms during a holiday dinner some time ago. I w
        103         25         25   ARCHIVED CONTENT: As a service to our readers, Harvard Health Publishing provides access to our libr
        106         27         25   My ability to sit peacefully day after day and write about health or enjoy my family owes more than 
        142         28         25   Low-fat diets, move over. When it comes to lowering cholesterol, a “portfolio” diet that includes ch
          0         35         23 1. Hea

In [74]:
# ============================================================================
# VISUALIZE INDIVIDUAL ARTICLE KNOWLEDGE GRAPHS
# ============================================================================

!pip install pyvis --quiet

from pyvis.network import Network
import networkx as nx
import os

def visualize_article_kg(kg_data, output_file):
    """
    Create interactive HTML visualization for one article's KG
    """
    graph = kg_data['graph']
    article_id = kg_data['id']
    
    if len(graph.nodes()) == 0:
        print(f"  Article {article_id}: No graph to visualize")
        return None
    
    print(f"  Article {article_id}: {len(graph.nodes())} nodes, {len(graph.edges())} edges")
    
    # Create pyvis network
    net = Network(
        height='800px',
        width='100%',
        bgcolor='#ffffff',
        font_color='black',
        directed=True,
        notebook=False
    )
    
    # Set physics options
    net.set_options("""
    {
      "physics": {
        "enabled": true,
        "barnesHut": {
          "gravitationalConstant": -8000,
          "centralGravity": 0.3,
          "springLength": 95,
          "springConstant": 0.04
        },
        "minVelocity": 0.75
      },
      "nodes": {
        "font": {"size": 16}
      },
      "edges": {
        "font": {"size": 12},
        "arrows": {"to": {"enabled": true, "scaleFactor": 0.5}}
      }
    }
    """)
    
    # Add nodes with colors based on degree
    for node in graph.nodes():
        degree = graph.degree(node)
        size = 20 + degree * 5
        
        # Color based on degree
        if degree > 5:
            color = '#e74c3c'  # Red for highly connected
        elif degree > 2:
            color = '#3498db'  # Blue for medium
        else:
            color = '#95a5a6'  # Gray for low
        
        net.add_node(
            node, 
            label=node, 
            size=size, 
            title=f"{node}\nConnections: {degree}",
            color=color
        )
    
    # Add edges with labels
    for source, target, data in graph.edges(data=True):
        relation = data.get('relation', 'related_to')
        net.add_edge(source, target, label=relation, title=relation)
    
    # Save
    net.save_graph(output_file)
    
    return net

# Create visualizations directory
os.makedirs('/kaggle/working/kg_visualizations', exist_ok=True)

print("=" * 80)
print("CREATING INTERACTIVE VISUALIZATIONS")
print("=" * 80)

# Visualize all successful KGs
visualized = 0
for kg in all_article_kgs:
    if kg['num_edges'] > 0:  # Only visualize articles with graphs
        article_id = kg['id']
        output_file = f'/kaggle/working/kg_visualizations/article_{article_id}_graph.html'
        visualize_article_kg(kg, output_file)
        visualized += 1

print(f"\n✓ Created {visualized} interactive visualizations!")
print(f"  Location: /kaggle/working/kg_visualizations/")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 12.4 MB/s eta 0:00:00a 0:00:01
CREATING INTERACTIVE VISUALIZATIONS
  Article 0: 35 nodes, 23 edges
  Article 1: 22 nodes, 15 edges
  Article 2: 11 nodes, 9 edges
  Article 3: 31 nodes, 18 edges
  Article 4: 26 nodes, 21 edges
  Article 5: 23 nodes, 16 edges
  Article 6: 23 nodes, 18 edges
  Article 7: 20 nodes, 14 edges
  Article 8: 16 nodes, 11 edges
  Article 9: 21 nodes, 18 edges
  Article 10: 35 nodes, 23 edges
  Article 11: 22 nodes, 15 edges
  Article 12: 11 nodes, 9 edges
  Article 13: 31 nodes, 18 edges
  Article 14: 26 nodes, 21 edges
  Article 15: 23 nodes, 16 edges
  Article 16: 23 nodes, 18 edges
  Article 17: 20 nodes, 14 edges
  Article 18: 16 nodes, 11 edges
  Article 19: 21 nodes, 18 edges
  Article 20: 32 nodes, 22 edges
  Article 21: 23 nodes, 18 edges
  Article 22: 22 nodes, 15 edges
  Article 23: 5 nodes, 3 edges
  Article 24: 30 nodes, 16 edges
  Article 25: 16 nodes, 9 edges
  Article 26: 35 nodes, 20 edge